In [64]:
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [68]:
# Read data from excel file

xlsx = r"C:\Users\mfaraday01\Documents\Code\Aging Analysis\credit_debit_records.xlsx"
debit = pd.read_excel (xlsx, 
                     sheet_name="Debits"
                     )
credit = pd.read_excel (xlsx, 
                     sheet_name="Credit"
                     )

In [69]:
# Unpivot "debit"

debit_unpiv = debit.melt(id_vars=["Zone", "Region", "District"], var_name="Date", value_name="Balance")

In [70]:
# Change datatypes

debit_unpiv[["Zone", "Region", "District"]] = debit_unpiv[["Zone", "Region", "District"]].astype(str)
debit_unpiv['Date'] = pd.to_datetime(debit_unpiv['Date']).dt.date

In [82]:
# Add new calculated columns

debit_unpiv["Due date"] = debit_unpiv["Date"] + timedelta(days=7)
debit_unpiv["Overdue"] = (datetime.date(datetime.now()) - debit_unpiv["Due date"]).dt.days.astype('int16')